# POC1 vision

In [1]:
output_dir = "../models"
image_dir = "../car_dataset/small_trainning_set"
result_file = "../my_project/export/result.json"

In [2]:
import doctest 
doctest.testmod()

TestResults(failed=0, attempted=0)

In [3]:
from pydantic import BaseModel
from typing import List

class TaskData(BaseModel):
    image_url: str

class ResultValue(BaseModel):
    """
    A class representing the value of a task completion result in Label Studio
    i.e. the label or the labels of a given input
    """
    choices: List[str]

class CompletionResult(BaseModel):
    """
    A class representing a task completion result in Label Studio
    """
    from_name: str
    id: str
    to_name: str
    type: str
    value: ResultValue

class TaskCompletion(BaseModel):
    """
    A class representing a task completion in Label Studio
    """
    id: int
    lead_time: float
    result: List[CompletionResult]

class Task(BaseModel):
    """
    A class representing a Label Studio task
    """
    id: int
    completions: List[TaskCompletion]
    data: TaskData
    task_path: str

class ExportedResults(BaseModel):
    """
    A class representing exported results from Label Sudio
    """
    tasks: List[Task]

In [4]:
from typing import List, Tuple
import json
from pathlib import Path

class LabelStudioDataFetcher:
    """
    This class allows parsing the the result returned by LabelStudio
    and feed it into fastai classifier in order to train a model.
    
    Examples:
    --------
    >>> result_file = "../my_project/export/result.json"
    >>> fetcher = LabelStudioDataFetcher(result_file)
    >>> fetcher.path
    PosixPath('../my_project/export/result.json')
    >>> LabelStudioDataFetcher("../proje_im_procjcdlLG/")
    Traceback (most recent call last):
        ...
    FileNotFoundError: The file does not exist: ../proje_im_procjcdlLG/
    """
    def __init__(self, result_file: str):
        self.path = Path(result_file)
        if not self.path.exists():
            raise FileNotFoundError(f"The file does not exist: {result_file}")
        self.get_image_data()
        self.labels, self.filenames = self.get_labels_and_paths()
    
    def get_image_data(self) -> None:
        with open(self.path, "r") as content:
            self.tasks = json.load(content)
            
    def get_labels_and_paths(self) -> Tuple[List[str], List[Path]]:
        """
        Take a list of tasks as argument and return two lists:
            - labels: List of labels
            - filenames: List of filenames
        """
        list_of_tasks = ExportedResults(tasks=self.tasks)
        return zip(*[
            (
                task.completions[0].result[0].value.choices[0],
                Path(task.task_path)
            )
            for task in list_of_tasks.tasks
        ])

In [8]:
import os
from fastai.vision import ImageDataBunch, get_transforms, models, cnn_learner, accuracy

def fastai_image_classifier(image_dir, filenames, labels, output_dir):
    """
    This script provides FastAI-compatible training for the input labeled images
    :param image_dir: directory with images
    :param filenames: image filenames
    :param labels: image labels
    :param output_dir: output directory where results will be exported
    :return: fastai.basic_train.Learner object
    """
    tfms = get_transforms()
    data = ImageDataBunch.from_lists(
        Path(image_dir),
        filenames,
        labels=labels,
        ds_tfms=tfms,
        size=224,
        bs=4
    )
    learn = cnn_learner(data, models.resnet18, metrics=accuracy, path=output_dir)
    learn.fit_one_cycle(2) # Here the par 20 represents the number of epoch
    return learn

Now we are going to train our model using the function fastai_image_classifier:

In [9]:
fetcher = LabelStudioDataFetcher(result_file)
fetcher.get_image_data()
labels, filenames = fetcher.get_labels_and_paths()
learn = fastai_image_classifier(image_dir, filenames, labels, output_dir)

epoch,train_loss,valid_loss,accuracy,time
0,2.477970,2.144558,0.473684,00:16
1,1.956651,1.420742,0.368421,00:15


In [11]:
learn.data.classes

['Back', 'Discard', 'Front', 'Left', 'Right']

Let’s try this model: 😊 

In [ ]:
from fastai.vision import open_image

In [ ]:
learn.predict(open_image("../car_dataset/cars_test/00061.jpg"))

In [ ]:
learn.predict(open_image("../car_dataset/cars_test/00029.jpg"))

In [ ]:
learn.predict(open_image("../car_dataset/cars_test/00322.jpg"))

In [ ]:
learn.predict(open_image("../car_dataset/cars_test/00362.jpg"))

In [ ]:
learn.predict(open_image("../car_dataset/cars_test/ji.jpg"))

We got the right answer for the last tests.

We just have to call learn.export to save all the information of our Learner object for inference: the stuff we need in the DataBunch (transforms, classes, normalization...), the model with its weights and all the callbacks our Learner was using. Everything will be in a file named export.pkl in the folder learn.path.

For more informations, see: https://docs.fast.ai/tutorial.inference.html#A-classification-problem

In [ ]:
print(learn.path)

In [ ]:
learn.export()

Now the folder models contains the file export.pkl 

Let’s use this exported file in order to feed a new model: 

In [ ]:
from fastai.vision import load_learner
new_model = load_learner('../models')

In [ ]:
new_model.predict(open_image("../car_dataset/cars_test/00266.jpg"))

In [25]:
new_model.predict(open_image("../car_dataset/cars_test/00216.jpg"))

(Category Back,
 tensor(0),
 tensor([9.9872e-01, 3.6774e-05, 1.3523e-05, 2.9802e-04, 9.3096e-04]))

In [26]:
new_model.predict(open_image("../car_dataset/cars_test/00167.jpg"))

(Category Left,
 tensor(3),
 tensor([3.8665e-06, 8.7824e-07, 6.3761e-07, 9.9865e-01, 1.3466e-03]))

In [27]:
new_model.predict(open_image("../car_dataset/cars_test/00213.jpg"))

(Category Front,
 tensor(2),
 tensor([1.5775e-01, 9.6136e-04, 7.9509e-01, 4.5905e-02, 2.9117e-04]))

In [28]:
new_model.predict(open_image("../car_dataset/cars_test/sony.jpg"))

(Category Discard,
 tensor(1),
 tensor([9.7729e-05, 9.8653e-01, 5.9017e-04, 2.0625e-03, 1.0719e-02]))

Again, we got the right answer!

* References:

https://docs.fast.ai/basic_train.html#Learner

https://forums.fast.ai/t/how-to-make-predictions-with-vision-learner-solved/34456/15

https://docs.fast.ai/tutorial.inference.html#A-classification-problem

https://github.com/gdoteof/neuralnet_stuff/blob/master/kaggle_whales.ipynb

https://docs.fast.ai/vision.image.html#open_image